In [1]:
import os
import argparse
from tqdm import tqdm

import torch
import numpy as np
import librosa
import IPython.display as ipd
import matplotlib.pyplot as plt

from generate_samples_batch import Diffsound
from specvqgan.modules.losses.vggishish.transforms import Crop

/home/yoonjin/anaconda3/envs/ttss2/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/yoonjin/anaconda3/envs/ttss2/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE
  warn(f"Failed to load image Python extension: {e}")


In [2]:
class CropImage(Crop):
    def __init__(self, *crop_args):
        super().__init__(*crop_args)

In [3]:
def load_audio(path):
    spec = np.load(path)
    
    item = {}
    item['input'] = spec

    mel_num=80
    spec_crop_len=848
    transforms = CropImage([mel_num, spec_crop_len], False)

    item = transforms(item)
    image = 2 * item['input'] - 1 # why --> it also expects inputs in [-1, 1] but specs are in [0, 1]
    spec = image.astype(np.float32)
    
    return torch.tensor(spec)

In [4]:
device = "cuda:0"
replication = 10
model_path = '../../diffsound/diffsound_audioset_audiocaps.pth'

In [5]:
config_path = os.path.join('caps_text.yaml')
ckpt_vocoder = os.path.join('../vocoder/logs/vggsound/')
save_path = os.path.join('results', 'inference_test', os.path.basename(model_path))
os.makedirs(save_path, exist_ok=True)

In [6]:
# Get the model
diffsound = Diffsound(config=config_path, path=model_path, ckpt_vocoder=ckpt_vocoder)

caps_text.yaml
Restored from ../../diffsound/last.ckpt


/home/yoonjin/anaconda3/envs/ttss2/lib/python3.8/site-packages/torchvision/transforms/transforms.py:332: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(


{'overall': {'trainable': '494.72M', 'non_trainable': '0', 'total': '494.72M'}, 'content_codec': {'trainable': '68.61M', 'non_trainable': '0', 'total': '68.61M'}, 'condition_codec': {'trainable': '0', 'non_trainable': '0', 'total': '0'}, 'transformer': {'trainable': '426.11M', 'non_trainable': '0', 'total': '426.11M'}, 'first_stage_permuter': {'trainable': '0', 'non_trainable': '0', 'total': '0'}}
Model missing keys:
 []
Model unexpected keys:
 []
Evaluate EMA model


In [17]:
prompt = 'A man and woman speak and water splashes'
target_emb = diffsound.save_text_emb(prompt, save_path, 'prompt')
wav_path = diffsound.inference_generate_sample_with_text_emb(text_emb=target_emb, truncation_rate=0.85, 
                                                             replicate=1, save_root=save_path, 
                                                             inference_name=None, fast=False, name=prompt)

ipd.display(ipd.Audio(wav_path, rate=22050))
print(prompt)

100%|██████████| 1/1 [00:00<00:00,  2.47it/s]


A man and woman speak and water splashes
